In [32]:
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import requests
import base64
import io
import time
from PIL import Image
import re

# --- CONFIG -----
class ConfigLLaVAVision:
    ENDPOINT_ID = "og65lbckc1lf24"
    API_KEY = ""  # ⛔ Put your API key here!
    BASE_URL = f"https://api.runpod.ai/v2/{ENDPOINT_ID}"

# --- UTILITIES ---
def image_to_base64(img):
    buffered = io.BytesIO()
    img.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def get_structured_caption(image):
    prompt = (
    """You are a vision-language assistant. Your task is to describe an image using exactly five predefined categories. You MUST return your response in a **single line**, using the following key-value format:
    main_objects: ..., location: ..., action: ..., attributes: ..., background: ...
    Use only this structure. Do not add any extra commentary, greetings, explanations, line breaks, or bullet points.
    If a category is not clearly visible, write `none`.
    Output Format Example (follow this EXACTLY) the content is just an example:
    main_objects: A small brown cat, location: Sitting on a windowsill, action: Looking outside, attributes: Brown fur, small size, background: A cozy sunlit room
    Now analyze the image and output only the structured result in the correct format:"""
    )

    image_b64 = image_to_base64(image)
    payload = {
        "input": {
            "prompt": prompt,
            "source": image_b64,
            "stop": ["<|eot_id|>", "<|start_header_id|>"]  # Stop generation early
        }
    }

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {ConfigLLaVAVision.API_KEY}"
    }

    try:
        submit_response = requests.post(
            f"{ConfigLLaVAVision.BASE_URL}/run",
            json=payload,
            headers=headers
        )
        submit_response.raise_for_status()
        job_id = submit_response.json().get("id")

        if not job_id:
            return "[ERROR] No job ID received"

        # Poll job
        for _ in range(30):
            time.sleep(2)
            status_response = requests.get(
                f"{ConfigLLaVAVision.BASE_URL}/status/{job_id}",
                headers=headers
            )
            status_json = status_response.json()
            status = status_json.get("status")

            if status == "COMPLETED":
                return status_json.get("output", {}).get("text", "")
            elif status == "FAILED":
                return "[ERROR] Job failed"

        return "[ERROR] Timed out"

    except Exception as e:
        return f"[ERROR] {e}"

# --- PARSER ---
def parse_caption_to_dict(caption_str: str) -> dict:
    fields = ["main_objects", "location", "action", "attributes", "background", "time"]
    result = {}

    for field in fields:
        pattern = rf"{field}:\s*(.*?)(?=,\s*\w+:|$)"
        match = re.search(pattern, caption_str, re.IGNORECASE)
        if match:
            value = match.group(1).strip()
            if value.lower() != "none":
                result[field] = [v.strip() for v in value.split(",") if v.strip()]

    return result

def extract_caption_line(raw_caption: str) -> str:
    try:
        # Remove <|eot_id|> from the raw caption
        raw_caption = re.sub(r"<\|eot_id\|>", "", raw_caption)
        
        split_token = "<|start_header_id|>assistant<|end_header_id|>"
        if split_token in raw_caption:
            assistant_part = raw_caption.split(split_token)[-1]
            for line in assistant_part.splitlines():
                line = line.strip()
                if line.lower().startswith("main_objects:"):
                    return line
    except Exception as e:
        print(f"[Parse Error] {e}")
    return None

# --- USAGE EXAMPLE ---
if __name__ == "__main__":
    image_path = "/content/Pixar animation of 8 year old Xochitl Gomez with a.webp"
    image = Image.open(image_path)

    print("🔄 Sending image to LLaVA...")
    raw_caption = get_structured_caption(image)
    print("🧠 Raw caption output:", raw_caption)

    # Extract the model's answer
    caption_line = extract_caption_line(raw_caption)

    if caption_line:
        parsed = parse_caption_to_dict(caption_line)
        print("✅ Parsed Caption Dictionary:")
        print(parsed)
    else:
        print("⚠️ No valid caption found in response.")



🔄 Sending image to LLaVA...
🧠 Raw caption output: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>You are a vision-language assistant. Your task is to describe an image using exactly five predefined categories. You MUST return your response in a **single line**, using the following key-value format:
    main_objects:..., location:..., action:..., attributes:..., background:...
    Use only this structure. Do not add any extra commentary, greetings, explanations, line breaks, or bullet points.
    If a category is not clearly visible, write `none`.
    Output Format Example (follow this EXACTLY) the content is just an example:
    main_objects: A small brown cat, location: Sitting on a windowsill, action: Looking outside, attributes: Brown fur, small size, background: A cozy sunlit room
    Now analyze the image and output only the structured result in the correct format:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

main_objects: A girl, location: In front

In [ ]:
teacher {'main_objects': ['A white cat'], 'location': ['Sitting on a couch'], 'action': ['Sleeping'], 'attributes': ['White fur', 'large size'], 'background': ['A sunny room with a window.']}
student {'main_objects': ['A large orange and white cat'], 'location': ['Lying on a sofa'], 'action': ['Sleeping'], 'attributes': ['Orange and white fur', 'large size'], 'background': ['A sunny living room with a window.']}

In [ ]:
{'main_objects': ['A small brown cat'], 'location': ['Sitting on a windowsill'], 'action': ['Looking outside'], 'attributes': ['Brown fur', 'small size'], 'background': ['A cozy sunlit room<|eot_id|>']}

In [31]:
from sentence_transformers import SentenceTransformer, util

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define structured captions for teacher and student
teacher_caption = {
    "main_objects": ["white cat"],
    "location": ["Sitting on a couch"],
    "action": ["Sleeping"],
    "attributes": ["White fur", "large size"],
    "background": ["A sunny room with a window."]
}

student_caption = {
    "main_objects": ["A large orange and white cat"],
    "location": ["Lying on a sofa"],
    "action": ["Sleeping"],
    "attributes": ["Orange and white fur", "large size"],
    "background": ["A sunny living room with a window."]
}

# Define weights for each category
weights = {
    "main_objects": 0.3,
    "location": 0.2,
    "action": 0.25,
    "attributes": 0.15,
    "background": 0.1
}

# Function to compute semantic similarity
def compute_similarity(list1, list2):
    if not list1 or not list2:
        return 0.0
    text1 = ', '.join(list1)
    text2 = ', '.join(list2)
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    return float(util.cos_sim(embedding1, embedding2).item())

# Calculate weighted similarity
total_score = 0.0
details = {}
for key in teacher_caption:
    score = compute_similarity(teacher_caption[key], student_caption[key])
    weighted_score = score * weights[key]
    total_score += weighted_score
    details[key] = {
        "raw_similarity": round(score, 3),
        "weighted_contribution": round(weighted_score, 3)
    }

total_score = round(total_score, 3)
details["final_similarity_score"] = total_score

# Print result
import pprint
pprint.pprint(details)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'action': {'raw_similarity': 1.0, 'weighted_contribution': 0.25},
 'attributes': {'raw_similarity': 0.917, 'weighted_contribution': 0.138},
 'background': {'raw_similarity': 0.962, 'weighted_contribution': 0.096},
 'final_similarity_score': 0.842,
 'location': {'raw_similarity': 0.69, 'weighted_contribution': 0.138},
 'main_objects': {'raw_similarity': 0.734, 'weighted_contribution': 0.22}}
